In [1]:
import pandas as pd
import polars as pl
import numpy as np
from tqdm import tqdm

In [2]:
def setSeed(x):
    seed_counter = hex(x)[2:]
    seed_counter_binary = bin(x)[2:]

    if len(seed_counter_binary) < cells:
        seed_counter_binary = "0"*(cells-len(seed_counter_binary)) + seed_counter_binary

    for i in range(cells-1, -1, -1):
        row = i // grid_width
        col = i % grid_width
        life_grid[row, col] = (seed_counter_binary[i] == '1')

    state_buffer = np.zeros(shape=(grid_height, grid_width)).astype(int)
    return str.upper(seed_counter)

In [3]:
def evaluateState(x, y):
    for i in range(grid_height):
        for j in range(grid_width):
            counter = 0
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if (di == 0 and dj == 0):
                        continue
                    ni = i + di
                    nj = j + dj
                    if ni >= 0 and ni < grid_height and nj >= 0 and nj < grid_width:
                        counter += x[ni, nj]
            if x[i, j] == 1:
                y[i, j] = int(counter == 2 or counter == 3)
            else:
                y[i, j] = int(counter == 3)
    return y

In [4]:
def evaluateParams(life_grid, state_buffer):
    if np.all(life_grid == 0):
        lifetime = "0"
        period = "N/A"
        period_class = "N/A"
        return lifetime, period, period_class

    virtual_life_grid = life_grid.copy()
    virtual_state_buffer = state_buffer.copy()

    seen_map = {}
    counter = 0

    seen_map[virtual_life_grid.tobytes()] = counter

    while True:
        evaluateState(virtual_life_grid, virtual_state_buffer)
        counter += 1

        if not np.any(virtual_state_buffer):
            lifetime = str(counter)
            period = "N/A"
            period_class = "N/A"
            break

        state_key = virtual_state_buffer.tobytes()

        if state_key in seen_map:
            lifetime = f"Persistent from Epoch {counter} onwards"
            x = counter - seen_map[state_key]

            if x == 1:
                period_class = "Still Life"
            else:
                period_class = "Oscillator"

            period = str(x)
            break

        seen_map[state_key] = counter
        virtual_life_grid = virtual_state_buffer.copy()

    return lifetime, period, period_class

In [20]:
grid_height = 1
grid_width = 1

cells = grid_width * grid_height
life_grid = np.zeros(shape=(grid_height, grid_width)).astype(int)
state_buffer = np.zeros(shape=(grid_height, grid_width)).astype(int)

df = pd.DataFrame(columns = ["Seed", "Lifetime", "Period", "Period Class"])
for i in tqdm(range(2**cells)):
    seed = str(grid_height)+"x"+str(grid_width)+"x"+setSeed(i)
    lifetime, period, period_class = evaluateParams(life_grid, state_buffer)
    df.loc[len(df)] = [seed, lifetime, period, period_class]
t = df[~df["Lifetime"].str.contains("Persistent")].drop(columns=["Period", "Period Class"])
p = df[df["Lifetime"].str.contains("Persistent")]
del df

if t.shape[0] > 0:
    t["Lifetime"] = t["Lifetime"].astype(int)
    t.to_csv(f"{grid_height}x{grid_width}_t.csv", index=False)

if p.shape[0] > 0:
    p["Period"] = p["Period"].astype(int)
    p.to_csv(f"{grid_height}x{grid_width}_p.csv", index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1258.23it/s]
